In [1]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[-0.1429],
        [-0.0247]], grad_fn=<AddmmBackward0>)

In [2]:
"""参数访问"""
print(net[2].state_dict()) ## net[i]拿到的是对应序列i的模块block

OrderedDict([('weight', tensor([[ 0.3316, -0.0687, -0.1680, -0.2119,  0.3420, -0.0318, -0.2327,  0.2489]])), ('bias', tensor([-0.0736]))])


In [3]:
"""目标参数"""
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.0736], requires_grad=True)
tensor([-0.0736])


In [4]:
net[2].weight.grad == None

True

In [35]:
"""一次性访问所有参数"""
print(*[(name, param.shape, param.data) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])
net[0].named_parameters()

('weight', torch.Size([8, 4]), tensor([[-0.5752,  0.3208,  0.3112,  0.0353],
        [ 0.1366,  0.3840, -0.7018,  0.4492],
        [ 0.3155, -0.1341, -0.3769,  0.5234],
        [ 0.0316, -0.3087,  0.1451, -0.6890],
        [-0.2893, -0.2196,  0.5288, -0.0881],
        [ 0.4657, -0.0887,  0.1366,  0.4966],
        [-0.2594, -0.1668,  0.6835, -0.1681],
        [-0.5482, -0.0947,  0.4899,  0.0138]])) ('bias', torch.Size([8]), tensor([0., 0., 0., 0., 0., 0., 0., 0.]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


<generator object Module.named_parameters at 0x7facac3afb30>

In [6]:
net.state_dict()['2.bias'].data

tensor([-0.0736])

In [14]:
"""从嵌套块手机参数"""
def block1():
  return nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 4), nn.ReLU())

def block2():
  net = nn.Sequential()
  for i in range(4):
    net.add_module(f'block {i}', block1())
  return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[0.3375],
        [0.3375]], grad_fn=<AddmmBackward0>)

In [15]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [21]:
rgnet[0][0][2].weight.data

tensor([[ 0.2371, -0.1546, -0.1348, -0.0928,  0.3040,  0.1513, -0.2240, -0.1547],
        [-0.0568,  0.1286, -0.2953, -0.0942, -0.1726, -0.0738,  0.1667, -0.3012],
        [-0.0322,  0.1578,  0.0162, -0.0721,  0.2224, -0.0513,  0.2186,  0.3337],
        [-0.2613, -0.1571, -0.1570, -0.1131, -0.1295, -0.2694, -0.2706,  0.0908]])

In [24]:
"""内置初始化"""
def init_normal(m):
  if type(m) == nn.Linear: ## 这里可以使用isinstance进行比较
    nn.init.normal_(m.weight, mean=0, std=0.01)
    nn.init.zeros_(m.bias)
print(net[0].bias.shape)
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

torch.Size([8])


(tensor([ 0.0116,  0.0197, -0.0086, -0.0236]), tensor(0.))

In [26]:
def init_constant(m):
  if type(m) == nn.Linear:
    nn.init.constant_(m.weight, 1)
    nn.init.zeros_(m.bias)

net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [27]:
Y = torch.rand(3, 4)
net(Y)
net[0].bias.shape

torch.Size([8])

In [33]:
w = torch.arange(4, dtype=torch.float32).reshape((2,2))
x = torch.ones(2, 2, dtype=torch.float32)
b = torch.tensor([1,1], dtype= torch.float32)
w,x, torch.mm(w, x) + b

(tensor([[0., 1.],
         [2., 3.]]), tensor([[1., 1.],
         [1., 1.]]), tensor([[2., 2.],
         [6., 6.]]))

In [34]:
"""对某些块应用不同的初始化方法"""
def xavier(m):
  if type(m) == nn.Linear:
    nn.init.xavier_uniform_(m.weight)

def init_42(m):
  if type(m) == nn.Linear:
    nn.init.constant_(m.weight, 42)

net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([-0.5752,  0.3208,  0.3112,  0.0353])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [41]:
def my_init(m):
  if type(m) == nn.Linear:
    print('Init', *[(name, param.shape) for name, param in m.named_parameters()][0]) ## 这里0表示取weight，1表示取bias
    nn.init.uniform_(m.weight, -10, 10)
    m.weight.data *= m.weight.data.abs() >= 5 ## 将权重值不大于等于的5值置0

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[ 8.1215, -0.0000, -0.0000, -7.2906],
        [-0.0000, -0.0000,  7.3965, -0.0000]], grad_fn=<SliceBackward0>)

In [42]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000,  1.0000,  1.0000, -6.2906])

In [45]:
"""参数绑定"""
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), shared, nn.ReLU(), shared,
           nn.ReLU(), nn.Linear(8, 1))
net(X)
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])
